# Introduction

# Load data

In [1]:
import pandas as pd
df = pd.read_csv('Example/data.txt', sep='\t')

# Interaction Detection

## Select features

In [ ]:
# Load FeatureSelector
from NetBMI.Interaction import FeatureSelector
selector = FeatureSelector(core_name="LightGBM")

# Conduct feature selection from df
selected_features = selector.select(df)

## Stacking Model

In [ ]:
from NetBMI.Interaction import StackingModel
Model = StackingModel()
best_scores = Model.stacking_model_search(df, selected_features)

## Model Explanation

## Network Construction

In [ ]:
# Load NetworkConstructor
from NetBMI.Interaction import NetworkConstructor
network_constructor = NetworkConstructor(df, selected_features, cutoff = 1.5, core_name="LightGBM")

# Construct sub-network for each group
interactions = network_constructor.construct_network()
# Construct conmbined network
combined_graph = network_constructor.compose_all(interactions)
# Remove isolated nodes from the network
Graph_BMI = network_constructor.remove_isolated_nodes(combined_graph)
# Save to .graphml file
network_constructor.save_graph(Graph_BMI, './Example')

## Network Analysis

### Conduct calculation of centrality and adjacent matrix

In [ ]:
from NetBMI.Interaction import NetworkMetrics
metrics_calculator = NetworkMetrics(Graph_BMI)
metrics = metrics_calculator.compute_metrics()

### Plots

In [2]:
from NetBMI.Interaction import NetworkVisualizer
visualizer = NetworkVisualizer(Graph_BMI, metrics)

In [ ]:
visualizer.plot_adjacency_matrix(save_pdf=False, pdf_path=None)

In [ ]:
visualizer.plot_centrality_measures(save_pdf=False, pdf_path=None)

In [ ]:
visualizer.plot_network_communities(save_pdf=True, pdf_path=None)

# Staqtistic plots

## Single ROC

In [ ]:
from NetBMI.plot import plot_single_roc
AUCs = plot_single_roc("Example/data.txt")

## PCA

In [ ]:
from NetBMI.plot import plot_pca
plot_pca("Example/data.txt")